# User activity on Kiwi News

Since Kiwi News is token-gated for publishing and voting, most user data is already onchain.
We only need to look at the actions per month.

In [39]:
from pathlib import Path
from datetime import datetime
import jsonlines
import pandas as pd

In [40]:
tbl_activity = []
for p in sorted(Path("../data").glob("*.jsonl")):
    with jsonlines.open(p) as reader:
        for row in reader:
            dt = datetime.fromtimestamp(row["timestamp"])
            user = row['identity']
            tbl_activity.append([dt,user])
df_activity = pd.DataFrame.from_records(tbl_activity, columns=['timestamp','user'])
df_activity.head()


,timestamp,user
0,2023-01-01 00:00:01,0x0DF5Ba52e8C055950AaAf5fcFE829020e898ee60
1,2023-04-14 17:33:53,0xee324c588ceF1BF1c1360883E4318834af66366d
2,2023-04-14 21:58:55,0x78d32460D0a53Ac2678e869Eb6b4f6bA9d2Ef360
3,2023-04-14 22:29:09,0xee324c588ceF1BF1c1360883E4318834af66366d
4,2023-04-15 13:51:50,0xee324c588ceF1BF1c1360883E4318834af66366d


In [41]:
df_activity = df_activity.assign(year=lambda x: x.timestamp.dt.year, month=lambda x: x.timestamp.dt.month)
df_activity.head()

,timestamp,user,year,month
0,2023-01-01 00:00:01,0x0DF5Ba52e8C055950AaAf5fcFE829020e898ee60,2023,1
1,2023-04-14 17:33:53,0xee324c588ceF1BF1c1360883E4318834af66366d,2023,4
2,2023-04-14 21:58:55,0x78d32460D0a53Ac2678e869Eb6b4f6bA9d2Ef360,2023,4
3,2023-04-14 22:29:09,0xee324c588ceF1BF1c1360883E4318834af66366d,2023,4
4,2023-04-15 13:51:50,0xee324c588ceF1BF1c1360883E4318834af66366d,2023,4


Drop activity before September 2023.

In [42]:
df_activity = df_activity.drop(df_activity[(df_activity.year == 2023) & (df_activity.month < 9 )].index)

In [49]:
df_grouped = df_activity[['year','month', 'user']].groupby(by=['year','month', 'user']).value_counts()
df_grouped

year  month  user                                      
2023  9      0x0263D82a119a20f0A4992F11081B1B4746cA8c89    2
             0x035eBd096AFa6b98372494C7f08f3402324117D3    5
             0x0621ad7f4D70Bc625F2D90d3202521f8F6B9d3C2    2
             0x0DF5Ba52e8C055950AaAf5fcFE829020e898ee60    2
             0x0f8b68ae0940314E1cFe948831E3F7E1c28F0D54    3
                                                          ..
      12     0xC304Eef1023e0b6e644f8ED8f8c629fD0973c52d    4
             0xD01559BB03cc83d4661193b513e5A083Ad0cc8Cf    2
             0xFEdE2257Dc043f53a24Ac7947218BACfB0013B09    1
             0xe8BbE391340E0a3bf1aC13166c41266b65598CE9    5
             0xee324c588ceF1BF1c1360883E4318834af66366d    7
Name: count, Length: 244, dtype: int64

In [53]:
df_grouped.reset_index()

,year,month,user,count
0,2023,9,0x0263D82a119a20f0A4992F11081B1B4746cA8c89,2
1,2023,9,0x035eBd096AFa6b98372494C7f08f3402324117D3,5
2,2023,9,0x0621ad7f4D70Bc625F2D90d3202521f8F6B9d3C2,2
3,2023,9,0x0DF5Ba52e8C055950AaAf5fcFE829020e898ee60,2
4,2023,9,0x0f8b68ae0940314E1cFe948831E3F7E1c28F0D54,3
...,...,...,...,...
239,2023,12,0xC304Eef1023e0b6e644f8ED8f8c629fD0973c52d,4
240,2023,12,0xD01559BB03cc83d4661193b513e5A083Ad0cc8Cf,2
241,2023,12,0xFEdE2257Dc043f53a24Ac7947218BACfB0013B09,1
242,2023,12,0xe8BbE391340E0a3bf1aC13166c41266b65598CE9,5
